# NLP with Transformers
# Traitement du langage naturel avec des Transformers

Dans ce notebook, nous allons utiliser la library transformers de Hugging Face pour le traitement du langage naturel (NLP). Les modèles de langages les plus performants (GPT, Llama etc...) sont très couteux en terme d'espace mémoire et il est souvent impossible de les faire fonctionner sur un ordinateur portable. Nous allons donc utiliser des modèles de taille réduite (et donc moins performants) dans ce notebook.

## ChatBot

L'utilisation la plus commune des modèles de langages (LLM) est aujourd'hui le ChatBot ce qui consiste en un genre d'assistant virtuel qui va répondre à nos questions.   
Avec Hugging Face, vous pouvez avoir votre propre ChatBot en local de la manière suivante. 

Nous utilisons une version réduite de [BlenderBot](https://ai.meta.com/blog/blenderbot-3-a-175b-parameter-publicly-available-chatbot-that-improves-its-skills-and-safety-over-time/) de Meta (facebook/blenderbot-400M-distill).

### Implémentation

In [1]:
from transformers import pipeline

/home/aquilae/anaconda3/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chatbot = pipeline(task="conversational",model="facebook/blenderbot-400M-distill")

Ce ChatBot ne gère que l'anglais, il faudra donc lui poser des questions en anglais.

In [3]:
from transformers import Conversation
user_message = """What is the best french deep learning course?"""
conversation = Conversation(user_message)
conversation = chatbot(conversation)
print(conversation)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Conversation id: 44c34bd3-ea1b-44b6-bd54-9127133cc941
user: What is the best  french deep learning course?
assistant:  I'm not sure, but I do know that French is one of the most widely spoken languages in the world.



Comme vous le voyez, le modèle est assez mal entraîné, il ne sait pas que le meilleur cours de Deep Learning est celui-ci.

Si vous voulez poser une autre question, la commande suivante permet d'avoir la réponse à une question en une seule ligne de code. 

In [4]:
conversation=Conversation("What is the most tasty fruit?")
print(chatbot(conversation))

Conversation id: d258da22-78e4-4621-a0e1-90776454a595
user: What is the most tasty fruit?
assistant:  I would have to say watermelon. It is so juicy and juicy.



Par contre, si vous souhaitez continuer la conversation déjà entamée, il faut utiliser cette fonction.

In [13]:
# Il faut spécifier le rôle (user) et ajouter votre message dans la conversation déjà existante
conversation.add_message({"role": "user","content": """What else do you recommend?"""})
print(chatbot(conversation))

Conversation id: c3e1a64c-5b40-4808-8632-38d9df14ed9d
user: What is the most tasty fruit?
assistant:  I would have to say watermelon. It is so juicy and juicy.
user: What else do you recommend?
assistant:  I would say mangos are pretty good too. They are sweet and tangy.



Vous savez maintenant comment utiliser un ChatBot avec la library transformers de Hugging Face. 

## Traduction

Nous allons maintenant regarder comment implémenter un traducteur. Pour cela, nous utilisons le modèle [No Language Left Behind](https://ai.meta.com/research/no-language-left-behind/) de facebook (facebook/nllb-200-distilled-600M) qui permet de faire depuis n'importe quel langage. Par soucis de mémoire, nous utilisons une version réduite du modèle.  

### Implémentation

In [14]:
traducteur = pipeline(task="translation",model="facebook/nllb-200-distilled-600M") 

In [24]:
text = """Le meilleur cours de d'apprentissage profond est celui-ci."""
text_translated = traducteur(text,src_lang="fra_Latn",tgt_lang="eng_Latn")
print("Le texte en anglais : ", text_translated[0]["translation_text"])
text_translated = traducteur(text,src_lang="fra_Latn",tgt_lang="jpn_Jpan")
print("Le texte en japonais : ",text_translated[0]["translation_text"])

Le texte en anglais :  The best course of deep learning is this one.
Le texte en japonais :  深い学習の最高のコースはこれです


La traduction est très bonne (pour l'anglais en tout cas, pour le japonais je n'ai malheuresement pas l'expertise) !   
Vous pouvez également tester d'autres combinaisons de langage, il faut juste spécifier le bon code que l'on peut trouver sur cette [page](https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200).

## Résumé de texte

Une autre tâche utile dans le traitement du langage naturel est le résumé de texte. Le modèle doit être capable de rassembler les informations les plus importantes d'un texte. Pour cela, nous utilisons le modèle [BART](https://research.facebook.com/publications/bart-denoising-sequence-to-sequence-pre-training-for-natural-language-generation-translation-and-comprehension/) de facebook (facebook/bart-large-cnn).

In [25]:
resumeur=pipeline(task="summarization",model="facebook/bart-large-cnn")

In [34]:
text= "Troyes is a beautiful city. Troyes is a commune and the capital of the department of Aube in the Grand Est region of north-central France. It is located on the Seine river about 140 km (87 mi) south-east of Paris. Troyes is situated within the Champagne wine region and is near to the Orient Forest Regional Natural Park.Troyes had a population of 61,996 inhabitants in 2018. It is the center of the Communauté d'agglomération Troyes Champagne Métropole, which was home to 170,145 inhabitants."
summary = resumeur(text,min_length=10,max_length=100)
print("Le résumé du texte : ",summary[0]["summary_text"]) #["summary_text"]

Le résumé du texte :  Troyes is a commune and the capital of the department of Aube in the Grand Est region of north-central France. It is located on the Seine river about 140 km (87 mi) south-east of Paris. Troyes had a population of 61,996 inhabitants in 2018.


Le résumé n'est pas très bon car il s'agit d'un petit modèle mais il a quand même su ressortir les informations clés et enlever les informations "moins importantes".

## Embedding de phrase

Un aspect important du langage naturel que nous avons abordé dans le cours sur les NLP est l'*embedding*. Pour rappel, cela consiste à projetter nos *tokens* (mots ou caractères par exemple) dans un espace latent. Cela va permettre de faire une première étape de rapprochement de ces mots. Des mots proches comme chiens et chats vont être proches dans l'espace latent tandis que "chien" et "est" vont être éloignés.   
Nous pouvons utiliser ces *embeddings* pour calculer la similarité entre deux phrases. Pour cela, nous allons utiliser la library sentence_transformers qui permet d'extraire l'*embedding* à partir d'un modèle pré-entrainé.   

Nous utilisons le modèle 
[all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [36]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
model = SentenceTransformer("all-MiniLM-L6-v2")

Nous allons regarder la similarité entre des phrases différentes.

In [42]:
sentences1 = ['The cat is chasing the mouse','A man is watching the television','The latest movie is awesome']
sentences2 = ['The dog sleeps in the kitchen','A boy watches TV','The new movie is so great']
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2,convert_to_tensor=True)
cosine_scores = util.cos_sim(embeddings1,embeddings2)
for i in range(len(sentences1)):
  print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i],
                                                sentences2[i],
                                                cosine_scores[i][i]))

The cat is chasing the mouse 		 The dog sleep in the kitchen 		 Score: 0.0601
A man is watching the television 		 A boy watches TV 		 Score: 0.7207
The latest movie is awesome 		 The new movie is so great 		 Score: 0.7786


Comme vous pouvez le voir, les phrases proches en terme de sens ont des *embeddings* assez similaires. Ce modèle est donc intéressant pour extraire des *embeddings*.   
Avoir un bon modèle extracteur d'*embedding* est souvent une première étape dans un projet de traitement du langage naturel.  